##**カルマンフィルタ**  


### カルマンフィルタを利用したマウスの動きの追跡
もしあなたの手が大きく震えている場合、カルマンフィルタは震えがなかった場合のマウスの軌跡を推定するのに使うことができます。

### 前提条件
このページで紹介するカルマンフィルタを理解するためには、以下の内容について知っておくとよいでしょう。

- **カルマンフィルタの紹介**
- **カルマンフィルタとは？**
- **カルマンフィルタの例: 物体の軌跡推定**

カルマンフィルタについて数学的に議論する前に、カルマンフィルタを使って何ができるのかを見てみましょう。

カルマンフィルタを利用すると、物体を追跡する際に、これまでの軌跡に基づいて次の物体の位置を推定することができます。

例えば、以下の図のように、これまで $t = 0, 1, 2$ の時間順に軌跡を得たとしましょう。

では、$t = 3$ のとき、物体はどこにいると考えるのが最も妥当でしょうか？おそらく、緑色の四角が次の最も妥当な位置だと考えられるでしょう。

**図1. これまでの軌跡に基づいて $t = 3$ での位置が最も高い確率でどこか？**

これをもう少し応用した例が以下の動画で確認できます。

動画では、カルマンフィルタを利用したボールの軌跡の推定が示されています。赤い円の大きさが大きいほど、次のステップの予測が不確実であることを意味します。

### 正規分布に関する二つの演算: 畳み込みと積
カルマンフィルタは正規分布を利用してすべての状態と動作を記述します。

また、カルマンフィルタは「時間に沿った状態の変化」を扱うため、正規分布で表現された状態の変化について知っておく必要があります。

状態の変化は、畳み込み（convolution）と積（product）という二つの演算によって表現できます。（詳細な議論は後述します。）

### 二つの演算過程
二つの演算過程に関する詳細な証明は「Products and Convolutions of Gaussian Probability Density Functions」という文献を参考にしてください。ここでは、この演算の結果だけを確認します。

畳み込みと積の両方で二つの正規分布を扱うため、それぞれの正規分布関数を次のように定義します。

$$
N_1(x; \mu_1, \sigma_1^2) = \frac{1}{\sqrt{2\pi\sigma_1^2}} \exp\left(-\frac{(x - \mu_1)^2}{2\sigma_1^2}\right) \quad \tag{1}
$$

$$
N_2(x; \mu_2, \sigma_2^2) = \frac{1}{\sqrt{2\pi\sigma_2^2}} \exp\left(-\frac{(x - \mu_2)^2}{2\sigma_2^2}\right) \quad \tag{2}
$$

今後、正規分布を表現するときは $N$ という記号を使用し、$N(x; \mu_1, \sigma_1^2)$ と書かれている場合は、定義域が $x$ で、平均と分散が $\mu_1, \sigma_1^2$ であることを意味します。

### 二つの正規分布の畳み込み（convolution）
最初に確認する状態変化の演算は、二つの正規分布の畳み込み（convolution）です。

畳み込み（convolution）は、任意の二つの関数 $f(t)$ と $g(t)$ に対して次のように定義される演算です。

$$
f(t) \ast g(t) = \int_{-\infty}^{\infty} f(\tau) g(t - \tau) d\tau \quad \tag{3}
$$

畳み込みは、一つの関数ともう一つの関数を反転・移動させて掛け合わせ、その区間に対して積分し、新たな関数を得る数学的な演算です。

図で示すと、次のようにある関数に他の関数をスライドさせながら掛け合わせ、得られた結果を用いて新しい関数を得るものです。

**図2. 二つの正規分布の畳み込み（convolution）**

### 二つの正規分布の積（product）
二番目に使用する状態変化の演算は、二つの正規分布の積（product）です。

二つの正規分布を掛けると、どんな結果が得られるのでしょうか？驚くべきことに、二つの正規分布を掛けても結果は正規分布になります。

もし積の結果で得られた正規分布を $N_{\text{new}}(x; \mu_{\text{new}}, \sigma_{\text{new}}^2)$ とすると、$\mu_{\text{new}}$ と $\sigma_{\text{new}}^2$ は次のように決定されます。

$$
\mu_{\text{new}} = \frac{\mu_1 \sigma_2^2 + \mu_2 \sigma_1^2}{\sigma_1^2 + \sigma_2^2} \quad \tag{4}
$$

$$
\sigma_{\text{new}}^2 = \frac{1}{\frac{1}{\sigma_1^2} + \frac{1}{\sigma_2^2}} = \frac{\sigma_1^2 \sigma_2^2}{\sigma_1^2 + \sigma_2^2} \quad \tag{6}
$$

**図3. 二つの正規分布の積（product）**
続きの翻訳を以下に示します。

---

$\mu_{\text{new}}$ と $\sigma_{\text{new}}^2$ の式からわかることは次の通りです。

まず、$\mu_{\text{new}}$ は $\mu_1$ と $\mu_2$ の間に位置し、二つの正規分布のうち、分散が小さい方により近く位置します。

また、$\sigma_{\text{new}}^2$ の式は次のように書き換えることもできます。

$$
\sigma_{\text{new}}^2 = \frac{\sigma_1^2 \sigma_2^2}{\sigma_1^2 + \sigma_2^2} = \frac{\sigma_2^2 \sigma_1^2}{\sigma_1^2 + \sigma_2^2} \quad \tag{7}
$$

これにより、新しい $\sigma_{\text{new}}^2$ の値は、$\sigma_1^2$ または $\sigma_2^2$ のどちらよりも常に小さくなることがわかります。

### 位置推定と移動
カルマンフィルタでは、すべての「状態」を正規分布を用いて表現すると述べました。

カルマンフィルタでは、現在の位置と移動量をすべて正規分布を用いて表現できます。

このとき、正規分布を利用することで得られる利点は、「不確実性」を含めて現在の位置や移動量を表現できることです。

例えば、位置を測定しようとする物体が現在 $x = 0$ にいる可能性が高いと考えてみましょう。

ここで、異なる二つの方法で位置を測定した場合、不確実性が異なるなら、次のように正規分布の分散を変えて表現できます。

**図4. 不確実性（分散）が異なる二つの正規分布の比較**

つまり、青色で示された正規分布と比べて、オレンジ色で示された正規分布は、$x = 0$ に物体が存在するということについての不確実性がより高いと考えることができます。

一方、移動量も正規分布を使って表現できます。例えば、次のステップで $x = 4$ だけ右に移動する物体があるとしましょう。

しかし、正確に $x = 4$ だけ移動するかどうかには不確実性があるため、次のように正規分布で表現できます。

**図5. $x = 4$ だけ移動するが、不確実に移動することを正規分布で表現**

では、$x = 0$ にいた物体が毎ステップ $x = 4$ だけ右に移動すると、物体の位置と不確実性は毎ステップどのように変わるでしょうか？

**図6. $x = 0$ に位置していた物体を $x = 4$ だけ二度移動させたとき、各位置での不確実性は正規分布の分散値で表現**

上の図から、物体が移動するたびに不確実性が大きくなっていることが確認できます。

これは、物体の移動が「元の位置を表現していた正規分布」と「移動に関する正規分布」という二つの式を畳み込み（convolution）することによって得られるからです。

まとめると、物体の移動は物体の現在の位置と物体の移動に関する正規分布を畳み込みして表現でき、毎回畳み込みを行うたびに位置に関する不確実性は大きくなります。

（移動に関する正規分布に含まれる不確実性が加わっていくためです。）

### ベイズの定理: 更新ルール
ベイズの定理は条件付き確率に関する定理の一つで、次のように書かれます。

$$
P(H | E) = \frac{P(E | H) P(H)}{P(E)} \quad \tag{8}
$$

ここで、$H$ は仮説（Hypothesis）、$E$ は証拠（Evidence）を意味します。

ベイズの定理で重要な点は、事前確率と事後確率の関係です。

ベイズの定理は確率に対する視点を変えて理解することで、その核心を理解できるのです。

ベイジアンの観点では、確率とは「主張に対する信頼度」であり、ベイズの定理は既存の主張に対する信頼度を証拠を観察することで更新するルールを示しています。

**図7. ベイズの定理で重要な二つの確率（事前確率、事後確率）**

ベイズの定理を理解する際のもう一つの重要な点は、ベイズの定理の式の右辺の分母は、事後確率の値を「確率」として作るための正規化の過程に過ぎないということです。

つまり、ベイズの定理がいくら複雑に見えても、$P(E)$ は単純に確率の値に過ぎず、これは正規化のための過程に過ぎません。

したがって、ベイズの定理の式は次のように書き換えることもできます。

$$
P(H | E) = \frac{1}{Z} P(E | H) P(H) \quad \tag{9}
$$

つまり、事後確率は事前確率 ($P(H)$) と尤度 ($P(E | H)$) の積に比例する値であるという点です。
続きの翻訳を以下に示します。

---

### シンプルなカルマンフィルタの動作プロセス
カルマンフィルタは、予測（Predict）と更新（Update）の二つのプロセスを通じて動作します。

**図8. カルマンフィルタの動作ステージは、PredictとUpdateの二つで構成されています。**

#### Predict（予測）
Predictステージでは、現在の位置から次の位置への遷移を記述します。

つまり、直前の位置から現在のステップへ移動する操作です。ここで、現在の位置と移動の両方は、位置推定と移動に関する正規分布を使って表現され、移動の実行プロセスは畳み込み（convolution）を用いて行われます。

#### Update（更新）
Updateステージでは、Predictを通じて計算された予測された位置を更新します。

この際、ベイズの定理と正規分布の積（product）が使用されます。Predictステージで予測された確率分布（$P(H)$）と、実際に測定した位置に関する確率分布（$P(E | H)$）を掛け合わせることで、更新された現在位置に関する確率分布（$P(H | E)$）が計算できます。

このとき、正規分布の積を利用して更新された現在位置の確率分布（事後確率分布）を計算し、正規分布の積のプロセスで示したように、積の結果としての正規分布は継続的に分散が減少するため、一定の法則に従って予測すると仮定すれば、事後確率の分散値は特定の値に収束することができます。つまり、測定された正確な位置に予測できるようになります。

### シンプルなカルマンフィルタの動作プロセス図解

**図 y.**

---

### 参考文献
- **Tutorial: The Kalman Filter, Tony Lacey, MIT**
- **Understanding Kalman Filters, Melda Ulusoy, Mathworks**
- **Product and Convolutions of Gaussian Probability Density Functions**

